## Steps

All steps that are played by default are **preprocessing** steps, but you can also add **postprocessing** steps, that act on the results instead of the circuit.

The steps will be filtered according to their type automatically.

## Available Processing Steps

### Pre-processing
- **CliffordTDecomposition** (pennylane_calculquebec/processing/steps/base_decomposition.py)  
  Decomposes gates into the Clifford+T+RZ set.  
  Parameters: None.

  **Example**:

  ```python
  import pennylane as qml
  from pennylane.workflow import construct_tape
  from pennylane.drawer import tape_text, tape_mpl
  from pennylane_calculquebec.processing.steps import CliffordTDecomposition

  # 1. define device & QNode
  dev = qml.device("default.qubit", wires=1)
  @qml.qnode(dev)
  def circuit():
      qml.T(wires=0)
      qml.RZ(0.5, wires=0)
      return qml.expval(qml.PauliZ(0))

  # 2. build raw tape (user ops only)
  tape = construct_tape(circuit, level="top")()

  # 3. apply your CalculQuébec decomposition
  step       = CliffordTDecomposition()
  decomposed = step.execute(tape)

  # 4a. text drawing
  print(tape_text(decomposed, show_wire_labels=True))

  # 4b. matplotlib drawing
  fig, ax = tape_mpl(decomposed)
  ax.set_title("Clifford+T Decomposed Circuit")
  fig.savefig("decomposed_circuit.png")  # or plt.show()

  ```

- **DecomposeReadout** (pennylane_calculquebec/processing/steps/decompose_readout.py)  
  Converts observables into computational basis measurements.  
  Parameters: None.

  **Example**:

  ```python
  import pennylane as qml
  from pennylane.workflow import construct_tape
  from pennylane_calculquebec.processing.steps import DecomposeReadout

  # Build a circuit returning an observable
  dev = qml.device("default.qubit", wires=1)

  @qml.qnode(dev)
  def circuit():
      qml.Hadamard(wires=0)
      return qml.expval(qml.PauliZ(wires=0))

  # Construct the tape for the QNode
  tape = construct_tape(circuit)()

  # Apply DecomposeReadout to convert expectation to computational basis measurement
  step = DecomposeReadout()
  post = step.execute(tape)

  # Inspect the measurement operators on the new tape
  print(post.measurements)

  ```

- **GateNoiseSimulation** (pennylane_calculquebec/processing/steps/gate_noise_simulation.py)  
  Injects gate noise according to the MonarQ model.  
  Parameters: `machine_name`, `use_benchmark`.

  **Example**:

  ```python
  import pennylane as qml
  from pennylane_calculquebec.processing.steps import GateNoiseSimulation

  # Simple circuit without noise
  dev = qml.device("default.qubit", wires=1)
  @qml.qnode(dev)
  def circuit():
      qml.Hadamard(wires=0)
      return qml.probs(wires=0)

  # Inject gate noise using the Yamaska machine profile
  noise_step = GateNoiseSimulation("yamaska", use_benchmark=True)
  noisy_tape = noise_step.execute(circuit.qtape)

  # Execute the noisy circuit
  noisy_qnode = qml.QNode(lambda: None, dev)
  noisy_qnode.tape = noisy_tape
  print(noisy_qnode())  # probabilities with simulated gate errors
  ```

- **MonarqDecomposition** (pennylane_calculquebec/processing/steps/native_decomposition.py)  
  Converts operations to MonarQ native gates.  
  Parameters: None.

  **Example**:

  ```python
  import pennylane as qml
  from pennylane_calculquebec.processing.steps import MonarqDecomposition
  from pennylane.workflow import construct_tape
  from pennylane.drawer import tape_text

  # Circuit using non-native ops
  dev = qml.device("default.qubit", wires=2)
  @qml.qnode(dev)
  def circuit():
      qml.SWAP(wires=[0, 1])  # placeholder non-native
      return qml.expval(qml.PauliZ(0))

  # Convert to MonarQ native gates
  step = MonarqDecomposition()
  native = step.execute(construct_tape(circuit)())
  print(tape_text(native))  # only MonarQ primitive gates
  ```

- **IterativeCommuteAndMerge** (pennylane_calculquebec/processing/steps/optimization.py)  
  Optimizes the circuit by commuting, merging rotations, and cancellations.  
  Parameters: None.

  **Example**:

  ```python
  import pennylane as qml
  from pennylane_calculquebec.processing.steps import IterativeCommuteAndMerge

  # Circuit with redundant rotations
  dev = qml.device("default.qubit", wires=1)
  @qml.qnode(dev)
  def circuit():
      qml.RX(0.5, wires=0)
      qml.RX(-0.5, wires=0)  # cancelling rotation
      return qml.expval(qml.PauliX(0))

  # Optimize by merging and cancelling
  step = IterativeCommuteAndMerge()
  optimized = step.execute(circuit.qtape)
  print(qml.draw(optimized)())  # empty or minimal operations
  ```

- **ASTAR**, **ISMAGS**, **VF2** (pennylane_calculquebec/processing/steps/placement.py)  
  Placement algorithms to map logical qubits onto physical qubits.  
  Parameters: `machine_name`, `use_benchmark`, `q1_acceptance`, `q2_acceptance`, `excluded_qubits`, `excluded_couplers`.

  **Example (ASTAR)**:

  ```python
  from pennylane_calculquebec.processing.steps import ASTAR

  # Map 2-qubit CNOT on Yamaska topology
  step = ASTAR("yamaska", use_benchmark=False)
  # Provide dummy tape with two-qubit gate
  # ...existing code for generating 2-qubit tape...
  mapped = step.execute(tape)
  print(mapped.operations)
  ```

- **Swaps** (pennylane_calculquebec/processing/steps/routing.py)  
  Routing algorithm that inserts SWAPs to connect non-adjacent qubits.  
  Parameters: `machine_name`, `use_benchmark`, `q1_acceptance`, `q2_acceptance`, `excluded_qubits`, `excluded_couplers`.

  **Example**:

  ```python
  from pennylane_calculquebec.processing.steps import Swaps

  # Route a non-adjacent CNOT on Yamaska
  step = Swaps("yamaska", use_benchmark=True)
  mapped = step.execute(tape)  # tape with CNOT on distant wires
  print(mapped.operations)  # contains added SWAP gates
  ```

- **PrintTape**, **PrintWires** (pennylane_calculquebec/processing/steps/print_steps.py)  
  Debug steps that print the circuit or wires.  
  Parameters: None.

  **Example (PrintTape)**:

  ```python
  from pennylane_calculquebec.processing.steps import PrintTape

  step = PrintTape()
  tape = circuit.qtape
  # Prints each operation in the tape
  step.execute(tape)
  ```

### Post-processing
- **IBUReadoutMitigation** (pennylane_calculquebec/processing/steps/readout_error_mitigation.py)  
  Iterative Bayesian readout error mitigation.  
  Parameters: `machine_name`, `initial_guess`.

- **MatrixReadoutMitigation** (pennylane_calculquebec/processing/steps/readout_error_mitigation.py)  
  Matrix inversion readout error mitigation.  
  Parameters: `machine_name`.

- **ReadoutNoiseSimulation** (pennylane_calculquebec/processing/steps/readout_noise_simulation.py)  
  Simulates readout noise on the results.  
  Parameters: `machine_name`, `use_benchmark`.

- **PrintResults** (pennylane_calculquebec/processing/steps/print_steps.py)  
  Debug step that prints the results.  
  Parameters: None.

In [ ]:
from pennylane_calculquebec.processing.config import MonarqDefaultConfig
from pennylane_calculquebec.processing.steps import IBUReadoutMitigation

readout_error_mitigation = IBUReadoutMitigation("yamaska")

my_config = MonarqDefaultConfig("yamaska")
my_config.steps.append(readout_error_mitigation)
print(*my_config.steps, sep="\n")

You can create new preprocessing / postprocessing steps by overriding the PreProcessing / PostProcessing classes.

In [ ]:
# abstract steps + empty config
from pennylane_calculquebec.processing.interfaces import PreProcStep, PostProcStep
from pennylane_calculquebec.processing.config import ProcessingConfig

# default steps
from pennylane_calculquebec.processing.steps import (
    CliffordTDecomposition,
    ASTAR,
    Swaps,
    IterativeCommuteAndMerge,
    MonarqDecomposition,
)

In [ ]:
# toy preprocessing step for printing the circuit operations
class PrintCircuit(PreProcStep):
    def execute(self, tape):
        print(*tape.operations)
        return tape


# toy postprocessing step for printing the results
class PrintResults(PostProcStep):
    def execute(self, tape, results):
        print(results)
        return results

In [ ]:
# this custom config will print the circuit, transpile, print the transpiled circuit
# and then print the unmitigated results, followed by the mitigated results.
my_config = ProcessingConfig(
    PrintCircuit(),
    CliffordTDecomposition(),
    ASTAR("yamaska"),
    Swaps("yamaska"),
    IterativeCommuteAndMerge(),
    MonarqDecomposition(),
    PrintCircuit(),
    PrintResults(),
    IBUReadoutMitigation("yamaska"),
    PrintResults(),
)
print(*my_config.steps, sep="\n")

### let's try our config with custom steps

In [ ]:
import pennylane as qml
from pennylane_calculquebec.API.client import MonarqClient

# Change the values in the parentheses for your credentials
my_client = MonarqClient("your host", "your user", "your access token", "your project")

dev = qml.device(
    "monarq.default", client=my_client, processing_config=my_config, shots=1000
)

dev.circuit_name = "your circuit"
dev.project = "your project"


# a simple ghz circuit
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT([1, 2])
    return qml.counts()


# lets print the circuit
print(qml.draw(circuit)())

results = circuit()

# you don't have to print results, since they are printed as a post processing step!
# print(results)